## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_map

# Configure gmaps API key
gmaps.configure(api_key=g_map)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,State College,US,40.7934,-77.8600,75.99,51,95,1.01,overcast clouds
1,1,Barrow,US,71.2906,-156.7887,30.22,80,90,18.41,overcast clouds
2,2,Pangnirtung,CA,66.1451,-65.7125,35.60,87,20,0.00,few clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,71.76,81,91,11.54,overcast clouds
4,4,Upernavik,GL,72.7868,-56.1549,29.64,76,61,4.83,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Puerto Ayora,EC,-0.7393,-90.3518,71.76,81,91,11.54,overcast clouds
6,6,Changqing,CN,36.5575,116.7272,73.56,74,100,10.07,overcast clouds
8,8,Camacha,PT,33.0833,-16.3333,70.54,73,75,9.22,broken clouds
9,9,Gua Musang,MY,4.8823,101.9644,73.36,93,100,1.99,overcast clouds
15,15,Thompson,CA,55.7435,-97.8558,55.56,76,75,3.44,broken clouds
17,17,Ahipara,NZ,-35.1667,173.1667,58.68,84,100,11.16,overcast clouds
21,21,Hamakita,JP,34.8000,137.7833,66.06,71,0,5.35,clear sky
23,23,Busselton,AU,-33.6500,115.3333,50.09,58,100,15.93,overcast clouds
24,24,Port Hardy,CA,50.6996,-127.4199,51.80,100,75,5.75,light rain
25,25,Hobart,AU,-42.8794,147.3294,53.46,98,90,9.22,drizzle


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                299
City                   299
Country                291
Lat                    299
Lng                    299
Max Temp               299
Humidity               299
Cloudiness             299
Wind Speed             299
Current Description    299
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                291
City                   291
Country                291
Lat                    291
Lng                    291
Max Temp               291
Humidity               291
Cloudiness             291
Wind Speed             291
Current Description    291
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,71.76,overcast clouds,-0.7393,-90.3518,
6,Changqing,CN,73.56,overcast clouds,36.5575,116.7272,
8,Camacha,PT,70.54,broken clouds,33.0833,-16.3333,
9,Gua Musang,MY,73.36,overcast clouds,4.8823,101.9644,
15,Thompson,CA,55.56,broken clouds,55.7435,-97.8558,
17,Ahipara,NZ,58.68,overcast clouds,-35.1667,173.1667,
21,Hamakita,JP,66.06,clear sky,34.8000,137.7833,
23,Busselton,AU,50.09,overcast clouds,-33.6500,115.3333,
24,Port Hardy,CA,51.80,light rain,50.6996,-127.4199,
25,Hobart,AU,53.46,drizzle,-42.8794,147.3294,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_map
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel Not Found....skipping.")
        

Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.
Hotel Not Found....skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,71.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Changqing,CN,73.56,overcast clouds,36.5575,116.7272,Hanting Hotel Jinan Changqing West Jingshi Road
8,Camacha,PT,70.54,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
9,Gua Musang,MY,73.36,overcast clouds,4.8823,101.9644,Mines Inn Hotel
15,Thompson,CA,55.56,broken clouds,55.7435,-97.8558,Burntwood Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))